# 2017 Dataset

This dataset differs from the 2015 dataset in that the network data is stored as pcap files rather than csv files. The only problem with this dataset is that it contains no attack data. However, it will be useful as a training dataset. Pcap files are much more generalisable than csv files. The problem with the pcap files is that I need to clean the data myself! The pcap files have been converted to flows using Argus - it is these files that will be loaded here.

In [18]:
import os
import pandas as pd
import numpy as np
import pprint
from IPython.display import Markdown, display
from datetime import datetime
from datetime import timezone
import pickle

##local python file holding the paths to the directories I store the log files in
from directories_to_use import argus_text_files_dir, getTestingDir

### Load data into Dataframe
This is largely straightforward, however, the date is stored in the name of the text file rather than the actual records so need to extract that and add that to the records.

In [2]:
"""
Read Argus text files into one single dataframe
"""
def readDataIntoDataframe(argus_text_files_dir):
    first_time = True
    for t_file in os.listdir(argus_text_files_dir):
        if first_time:
            new_df = pd.read_csv(argus_text_files_dir + t_file)
            date = removeDateFromName(t_file)
            new_df["StartTime"] = new_df["StartTime"].apply(lambda x : date + x)
            new_df["LastTime"] = new_df["LastTime"].apply(lambda x : date + x)
            first_time = False
        else:
            temp_df = pd.read_csv(argus_text_files_dir + t_file)
            date = removeDateFromName(t_file)
            temp_df["StartTime"] = temp_df["StartTime"].apply(lambda x : date + x)
            temp_df["LastTime"] = temp_df["LastTime"].apply(lambda x : date + x)
            new_df = pd.concat([new_df, temp_df], ignore_index=True)
    return new_df

def removeDateFromName(filename):
    year = filename.split("_")[-1][0:4]
    month = filename.split("_")[-1][4:6]
    day = filename.split("_")[-1][6:8]
    full_date = day +"-"+ month +"-"+ year + " "
    return full_date

data2017_df = readDataIntoDataframe(argus_text_files_dir)
print(data2017_df.shape)

E:\ProgramData\Anaconda2\envs\SWaT\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (20,21,107) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
E:\ProgramData\Anaconda2\envs\SWaT\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (107) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


(2682823, 118)


### Clean data (i.e. remove columns that are unhelpful)
Just removing columns that only contain Nans or only contain one unique value

In [3]:
"""
Loops through all the columns in the dataframe and removes any that only contain nans
"""
def removeNanColumns(dataframe):
    print("Shape before: ", dataframe.shape)
    columns_removed = []
    for col in dataframe:
        unique_vals = dataframe[col].unique()
        if unique_vals.shape[0] == 1:
            if np.isnan(unique_vals[0]):
                dataframe = dataframe.drop([col], axis=1)
                columns_removed.append(col)
    print("Shape after: ", dataframe.shape)
    print("Columns with only Nans: " + str(columns_removed))
    return dataframe

"""
Remove columns that only contain one unique value.
"""
def removeSingleColumns(dataframe):
    print("Shape before: ", dataframe.shape)
    columns_removed = []
    for col in dataframe:
        unique_vals = dataframe[col].unique()
        if unique_vals.shape[0] == 1:
            dataframe = dataframe.drop([col], axis=1)
            columns_removed.append(col)
    print("Shape after: ", dataframe.shape)
    print("Columns with only one unique value: " + str(columns_removed))
    return dataframe
"""
Prints the columns in a dataframe and all its unique values
"""
def printColumnsAndUniqueVals(dataframe):
    pp = pprint.PrettyPrinter(indent=4)
    for col in dataframe.columns:
        printmd("**" + col + "**: " + str(dataframe[col].unique()))
        
"""
Basically prints in markdown form, can also render HTML
"""
def printmd(string):
    display(Markdown(string))

data2017_df = removeNanColumns(data2017_df)
data2017_df = removeSingleColumns(data2017_df)
                
            

Shape before:  (2682823, 118)
Shape after:  (2682823, 70)
Columns with only Nans: ['SrcMac', 'DstMac', 'SrcOui', 'DstOui', 'sCo', 'dCo', 'sMpls', 'dMpls', 'sAS', 'dAS', 'iAS', 'NStrok', 'sNStrok', 'dNStrok', 'SIntPkt', 'SIntDist', 'SIntPktAct', 'SIntActDist', 'SIntPktIdl', 'SIntIdlDist', 'DIntPkt', 'DIntDist', 'DIntPktAct', 'DIntActDist', 'DIntPktIdl', 'DIntIdlDist', 'SrcJitter', 'SrcJitAct', 'DstJitter', 'DstJitAct', 'Label', 'srcUdata', 'dstUdata', 'sVlan', 'dVlan', 'sVid', 'dVid', 'sVpri', 'dVpri', 'SRange', 'ERange', 'sPktSz', 'sMaxPktSz', 'dPktSz', 'dMaxPktSz', 'sMinPktSz', 'dMinPktSz', 'dMinPktSz.1']
Shape before:  (2682823, 70)
Shape after:  (2682823, 59)
Columns with only one unique value: ['Trans', 'StdDev', 'AutoId', 'TotAppByte', 'SAppBytes', 'DAppBytes', 'PCRatio', 'Retrans', 'SrcRetra', 'DstRetra', 'pRetran']


In [4]:
"""Print columns and the percentage of nans present in each column"""
data2017_df.isnull().mean() * 100

StartTime      0.000000
LastTime       0.000000
Flgs           0.000000
Seq            0.000000
Dur            0.000000
RunTime        0.000000
IdleTime       0.000000
Mean           0.000000
Sum            0.000000
Min            0.000000
Max            0.000000
SrcAddr        0.000000
DstAddr        0.000000
Proto          0.000000
Sport         10.424169
Dport         10.424169
sTos          10.826022
dTos          38.928733
sDSb          10.826022
dDSb          38.928733
sTtl          10.826022
dTtl          38.928733
sHops         10.826022
dHops         38.928733
sIpId         10.826022
dIpId         38.928733
Cause          0.000000
TotPkts        0.000000
SrcPkts        0.000000
DstPkts        0.000000
TotBytes       0.000000
SrcBytes       0.000000
DstBytes       0.000000
Load           0.000000
SrcLoad        0.000000
DstLoad        0.000000
Loss           0.000000
SrcLoss        0.000000
DstLoss        0.000000
pLoss          0.000000
SrcGap        43.607573
DstGap        43

## Calculated changes made to the dataset
### Drop Columns
- **Inode** is an IP Address, so very difficult to replace. Also >90% of the values were missing - so will drop it.
- **TcpOpt** is 99% Nan so will drop that column
- **SrcTCPBase** and **DstTCPBase** are 43% missing and they refer to the base sequence number in a TCP transaction. As they are specifically related to TCP, it's not a surprise that there would be values missing since not all the traffic is TCP related. Can't replace with mean as not all traffic is TCP - if I replace with 0, it will skew data. Will have to remove the columns 
- **SrcWin** and **DstWin** seem to refer to jitter. They are quite useful but with 43% of values being Nans means that it might be misleading to include them. Will have to remove them. 
- **dTos** and **dDSb** can be removed as their only unique values are 0 and nan.
- **sIpId** and **dIpId** are not necessary. It is unique IDs that tie src, dst and port. As they contain nulls and are difficult to simply impute. They can be removed since we have all the individual elements that make up the IDs.
- **sVid** and **dVid** can be removed since sVLan and dVLan are the same thing, which is VLAN ID.

### Drop Rows
- **SPort** and **Dport** can not be safely replaced/imputed without causing inconsistencies. Will remove rows that lack these fields - this makes up 10% of the dataset
- **sHops**, **sTtl** and **sDSb** contains only 0.5% (after removing rows from above) of nan values to might as well remove rows containing Nans for this.

### Replace with 0
- **SrcGap** and **DstGap** is 43% nan but refers to bytes missing from the stream, therefore, when Nan, we can just replace with 0. 
- **sTos** refers to Type of Service (whether traffic should take precedence etc.). Nans can be set to 0 for these fields.

### Imputed
- **dHops** will need to be imputed. Values are either 0 or 1.
- **dTtl** will also need to be imputed.
- **TODO** currently going to remove the columns as they contain 40% Nans. Will attempt to impute them at a later stage and determine if it affects the classification result. 


In [5]:
"""
Drop columns with too many nulls
"""
def dropChosenColumns(dataframe, column_names):
    for column in column_names: 
        dataframe = dataframe.drop([column], axis=1)
    return dataframe
        
"""
Replace Nans with 0
"""
def replaceNansWithZero(dataframe, column_names):
    for column in column_names:
        dataframe[column] = dataframe[column].fillna(0)
    return dataframe

"""
Remove rows with nans for chosen columns
"""
def removeNanRows(dataframe, column_names):
    for column in column_names:
        dataframe = dataframe[dataframe[column].notna()]
    return dataframe

data2017_df = dropChosenColumns(data2017_df, ['TcpOpt', 'Inode', 'SrcWin', 'DstWin', 'dTos', 'dDSb', 'SrcTCPBase', 'DstTCPBase', 'dIpId', 'dHops', 'dTtl'])
data2017_df = removeNanRows(data2017_df, ['Sport', 'Dport', 'sIpId', 'sHops', 'sTtl', 'sDSb'])
data2017_df = replaceNansWithZero(data2017_df, ['SrcGap', 'DstGap', 'sTos'])
data2017_df.shape

(2391347, 48)

## Convert all data to numeric form so that it can be passed to ML classifiers
- **StartTime** and **LastTime** need to be converted to timestamps - though they will not be included in the classification
- **SrcAddr** and **DstAddr** need to be converted to unique values. 
- **Sport** and **Dport** needs to be cast from string and checked for hex (which would also have to be cast).
- **Flgs**, **Dir**, **Cause**, and **State** need to be converted to integers.


In [6]:
"""
Needs to know the category e.g. IPs and value to add to the dict of dicts (unique_vals)
Returns: the unique int assigned to the value
"""
def convertToNum(category, val, unique_vals):
    if category not in unique_vals:
        unique_vals[category] = {}
    if val.strip() not in unique_vals[category].keys():
        new_val = len(unique_vals[category].keys())
        unique_vals[category][val.strip()] = len(unique_vals[category].keys())
        return new_val
    else:
        return unique_vals[category][val.strip()]
    
"""
Port numbers are a special case. They're integers stored as strings.
They are either hex numbers or standard int strings - therefore, we 
need to check for hex before casting.
"""  
def convertPortToNum(val):
    if type(val) == str:
        isHex = '0x' in val
    elif type(val) == float:
        isHex = False
    if isHex:
        return int(val, base=16)
    else:
        return int(val)
    
    

"""
Takes a string in the following format:
14-06-2017 11:25:58.288831
Returns: Timestamp
"""
def createTimestamp(datetime_string):
    row_date = datetime.strptime(datetime_string, "%d-%m-%Y %H:%M:%S.%f")
    timestamp = row_date.replace(tzinfo=timezone.utc).timestamp()
    return timestamp

unique_vals = dict()

data2017_df["Sport"] = data2017_df["Sport"].apply(lambda x : convertPortToNum(x))
data2017_df["Dport"] = data2017_df["Dport"].apply(lambda x : convertPortToNum(x))
data2017_df["StartTime"] = data2017_df["StartTime"].apply(lambda x: createTimestamp(x))
data2017_df["LastTime"] = data2017_df["LastTime"].apply(lambda x: createTimestamp(x))
data2017_df["SrcAddr"] = data2017_df["SrcAddr"].apply(lambda x : convertToNum("ips", x, unique_vals))
data2017_df["DstAddr"] = data2017_df["DstAddr"].apply(lambda x : convertToNum("ips", x, unique_vals))
data2017_df["Cause"] = data2017_df["Cause"].apply(lambda x : convertToNum("cause", x, unique_vals))
data2017_df["State"] = data2017_df["State"].apply(lambda x : convertToNum("state", x, unique_vals))
data2017_df["Flgs"] = data2017_df["Flgs"].apply(lambda x : convertToNum("flgs", x, unique_vals))
data2017_df["Dir"] = data2017_df["Dir"].apply(lambda x : convertToNum("dir", x, unique_vals))
print(data2017_df.shape)


(2391347, 48)


## Load in malicious data from 2019 Dataset

In [7]:
data2019_df = readDataIntoDataframe(getTestingDir())
print(data2019_df.shape)

(4503102, 118)


## Reduce Feature set so that both datasets have the same amount of features

In [8]:
"""
Takes in two dataframes and makes sure that the columns
of test_df are the same as those of train_df. Returns a
modified dataframe.
"""
def alignToTrainingData(train_df, test_df):
    for col in test_df.columns:
        if col not in train_df.columns:
            test_df = test_df.drop([col], axis=1)
    return test_df

#data2019_mod_df = data2019_df.copy(deep=True)
data2019_df = alignToTrainingData(data2017_df, data2019_df)
print(data2019_df.shape)

(4503102, 48)


### Check for any remaning Nans

In [9]:
#pd.options.display.max_rows = 4000
data2019_df.isnull().mean() * 100

StartTime     0.000000
LastTime      0.000000
Flgs          0.000000
Seq           0.000000
Dur           0.000000
RunTime       0.000000
IdleTime      0.000000
Mean          0.000000
Sum           0.000000
Min           0.000000
Max           0.000000
SrcAddr       0.000000
DstAddr       0.000000
Proto         0.000000
Sport         0.529946
Dport         0.529946
sTos          0.438520
sDSb          0.438520
sTtl          0.438520
sHops         0.438520
sIpId         0.438520
Cause         0.000000
TotPkts       0.000000
SrcPkts       0.000000
DstPkts       0.000000
TotBytes      0.000000
SrcBytes      0.000000
DstBytes      0.000000
Load          0.000000
SrcLoad       0.000000
DstLoad       0.000000
Loss          0.000000
SrcLoss       0.000000
DstLoss       0.000000
pLoss         0.000000
SrcGap        1.956163
DstGap        1.956163
Rate          0.000000
SrcRate       0.000000
DstRate       0.000000
Dir           0.000000
State         0.000000
TcpRtt        0.000000
SynAck     

### Remove any rows that contain Nans (provided the percentage of nans is not large <2%)

In [10]:
data2019_df = removeNanRows(data2019_df, ['SrcGap', 'DstGap', 'Sport', 'Dport', 'sIpId', 'sHops', 'sTtl', 'sDSb'])

### Convert all non-numeric fields to numbers

In [11]:
data2019_df["Sport"] = data2019_df["Sport"].apply(lambda x : convertPortToNum(x))
data2019_df["Dport"] = data2019_df["Dport"].apply(lambda x : convertPortToNum(x))
data2019_df["StartTime"] = data2019_df["StartTime"].apply(lambda x: createTimestamp(x))
data2019_df["LastTime"] = data2019_df["LastTime"].apply(lambda x: createTimestamp(x))
data2019_df["SrcAddr"] = data2019_df["SrcAddr"].apply(lambda x : convertToNum("ips", x, unique_vals))
data2019_df["DstAddr"] = data2019_df["DstAddr"].apply(lambda x : convertToNum("ips", x, unique_vals))
data2019_df["Cause"] = data2019_df["Cause"].apply(lambda x : convertToNum("cause", x, unique_vals))
data2019_df["State"] = data2019_df["State"].apply(lambda x : convertToNum("state", x, unique_vals))
data2019_df["Flgs"] = data2019_df["Flgs"].apply(lambda x : convertToNum("flgs", x, unique_vals))
data2019_df["Dir"] = data2019_df["Dir"].apply(lambda x : convertToNum("dir", x, unique_vals))

### Add a new column representing whether the traffic is malicious or benign. 
0 is benign and 1 is malicious <br>
Go through time period of attack and set the value to 1. <br>
**Note** the time period was adjusted due to different time zones

In [12]:
data2019_df["Classification"] = 0
data2017_df["Classification"] = 0

In [13]:
first_attack = (data2019_df["StartTime"] >= createTimestamp("06-12-2019 02:20:00.00")) & (data2019_df["StartTime"] < createTimestamp("06-12-2019 03:30:00.00"))
data2019_df.loc[first_attack, "Classification"] = 1

second_attack = (data2019_df["StartTime"] >= createTimestamp("06-12-2019 04:30:00.00")) & (data2019_df["StartTime"] <= createTimestamp("06-12-2019 05:45:00.00"))
data2019_df.loc[second_attack, "Classification"] = 1

### Convert to numpy arrays and save
This will allow us to actually only load the arrays and unloading everything else (should help with memory issues)

In [14]:
data2019_arr = data2019_df.to_numpy()
data2017_arr = data2019_df.to_numpy()

In [16]:
np.save('data2019_arr.npy', data2019_arr)
np.save('data2017_arr.npy', data2017_arr)

In [19]:
"""
This code is taken straight from: https://stackoverflow.com/questions/19201290/how-to-save-a-dictionary-to-a-file/32216025
"""

def save_obj(obj, name ):
    with open('obj/'+ name + '.pkl', 'wb+') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [20]:
save_obj(unique_vals, "unique_vals")